# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096807


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:07<01:38,  3.52s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<01:04,  2.48s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:44,  1.77s/it]

Rendering models:  20%|██        | 6/30 [00:07<00:30,  1.27s/it]

...

Rendering models:  80%|████████  | 24/30 [00:09<00:00,  6.81it/s]

Rendering models:  87%|████████▋ | 26/30 [00:09<00:00,  7.24it/s]

Rendering models:  93%|█████████▎| 28/30 [00:09<00:00,  7.96it/s]

Rendering models: 100%|██████████| 30/30 [00:09<00:00,  9.07it/s]

not-logged-in-3b0b929f4475fd276561    0.025884
Jnursssmith5263                       0.001660
ddemarco5                             0.144230
jnarayanbvg                           0.002143
awright5                              0.002096
RTitchenal                            0.046198
framosuw                              0.000505
jaksonA                               1.083307
not-logged-in-6ead6661c7dcef165774    0.055707
lsautter                              0.000407
not-logged-in-777c491ddad2ebcf9bb1    0.024996
Jessica_Korkmaz                       0.010728
jmartinez468                          0.003697
acapirala                             0.000910
alexmnelson18                         0.000425
not-logged-in-fcfcbc88feb1aa0ed0a7    0.000633
Lavadude                              0.005381
not-logged-in-e991c008411b56a8790d    0.014949
ShirleyL                              0.000372
equidad1                              0.008420
ZoshiePoshie                          0.000841
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())